In [128]:
import streamlit  as st
import json 
import os
import pandas as pd
import numpy as np
import mysql.connector as mc
connection = mc.connect(user= 'root',password='ss3112',host = 'localhost')
cursor = connection.cursor()
cursor.execute('CREATE DATABASE phonepe_pulse')
cursor.execute('USE phonepe_pulse')

In [129]:
#                          1. This is to direct the path to get the data as states

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


path = "pulse/data/aggregated/transaction/country/india/state/"
user_state_list = os.listdir(path)

# Agg_state_list--> to get the list of states in India


#-----------------------------------------------------------------------------------------------------------------------

# This is to extract the data's to create a dataframe

clm = {'State': [], 'Year': [], 'Quater': [], 'Transaction_type': [],
       'Transaction_count': [], 'Transaction_amount': []}
for i in user_state_list:
    p_i = path+i+"/"
    Agg_yr = os.listdir(p_i)
    for j in Agg_yr:
        p_j = p_i+j+"/"
        Agg_yr_list = os.listdir(p_j)
        for k in Agg_yr_list:
            p_k = p_j+k
            Data = open(p_k, 'r')
            D = json.load(Data)
            try:
                for z in D['data']['transactionData']:
                    Name = z['name']
                    count = z['paymentInstruments'][0]['count']
                    amount = z['paymentInstruments'][0]['amount']
                    clm['Transaction_type'].append(Name)
                    clm['Transaction_count'].append(count)
                    clm['Transaction_amount'].append(amount)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quater'].append(int(k.strip('.json')))
            except:
                pass

Agg_Transac = pd.DataFrame(clm)
# Succesfully created a dataframe
#Agg_Transac.to_csv('phonepe_csv/Agg_Transac.csv',index = False)
#Agg_Transac_csv = pd.read_csv('phonepe_csv/Agg_Transac.csv')
Agg_Transac

,State,Year,Quater,Transaction_type,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
3589,west-bengal,2022,4,Peer-to-peer payments,184380244,6.202222e+11
3590,west-bengal,2022,4,Merchant payments,171667404,1.408077e+11
3591,west-bengal,2022,4,Recharge & bill payments,48921147,2.602663e+10
3592,west-bengal,2022,4,Financial Services,268388,2.611229e+08


In [130]:
cursor.execute("create table aggregate_transaction(state varchar(45), year int, Quater int, tansaction_type varchar(45), transaction_count int, transaction_amount float)")
for record in Agg_Transac.values:
    cursor.execute("insert into aggregate_transaction(state, year, Quater, tansaction_type, transaction_count, transaction_amount) values{}".format(tuple(record)))
    connection.commit()
print('Data inserted into Table-1 successfully')
    

#**************************************** ------  1 st finished  ------************************************************
    
    

Data inserted into Table-1 successfully


In [131]:
#                         2.  This is to direct the path to get the data as users per states:

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


path = "pulse/data/aggregated/user/country/india/state/"
user_state_list = os.listdir(path)


#---------------------------------------------------------------------------------------------------------------------


# This is to extract the data's to create a dataframe

clm = {'State': [], 'Year': [], 'Quater': [], 'Brand': [],
    'Brand_count': [], 'Brand_percentage': []}
for i in user_state_list:
    p_i = path+i+"/"
    year = os.listdir(p_i)
    for j in year:
        p_j = p_i+j+"/"
        file = os.listdir(p_j)
        for k in file:
            p_k = p_j+k
            Data = open(p_k, 'r')
            D = json.load(Data)
            try:
                for z in D['data']["usersByDevice"]:
                    
                    brand = z['brand']

                    brand_count = z['count']
                    brand_percentage = z["percentage"]
                    clm['Brand'].append(brand)
                    clm['Brand_count'].append(brand_count)
                    clm['Brand_percentage'].append(brand_percentage)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quater'].append(int(k.strip('.json')))
            except:
                pass 
userby_device = pd.DataFrame(clm)
# Data frame is created successfully and now we can save df in csv format
userby_device.to_csv("phonepe_csv/userby_device.csv")
userby_device


,State,Year,Quater,Brand,Brand_count,Brand_percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.015056
6728,west-bengal,2022,1,Infinix,284678,0.012987
6729,west-bengal,2022,1,Asus,280347,0.012790
6730,west-bengal,2022,1,Apple,277752,0.012671


In [132]:
cursor.execute("create table userby_device(state varchar(45),year int,Quarter int,users_brand varchar(45),users_count int,user_percentage float)")
for record in userby_device.values:
    cursor.execute("insert into userby_device(state, year, Quarter, users_brand, users_count, user_percentage) values{}".format(tuple(record)))
    connection.commit()
print('Data inserted into Table-2 successfully')
    

#**************************************** ------  2 nd finished  ------************************************************



Data inserted into Table-2 successfully


In [133]:
#                                  3.   This is for map_transaction:

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


path = "pulse/data/map/transaction/hover/country/india/state/"
state_list = os.listdir(path)
#Agg_state_list
# Agg_state_list--> to get the list of states in India


#-----------------------------------------------------------------------------------------------------------------------

# This is to extract the data's to create a dataframe

clm = {'State': [], 'Year': [], 'Quater': [], 'District': [],
    'Transaction_count': [], 'Transaction_amount': []}
for i in state_list:
    p_i = path+i+"/"
    year = os.listdir(p_i)
    for j in year:
        p_j = p_i+j+"/"
        file = os.listdir(p_j)
        for k in file:
            p_k = p_j+k
            Data = open(p_k, 'r')
            D = json.load(Data)
            try:
                for z in D['data']["hoverDataList"]:
                    district = z['name']
                    transaction_count = z['metric'][0]['count']
                    transaction_amount = z['metric'][0]['amount']
                    clm['District'].append(district)
                    clm['Transaction_count'].append(transaction_count)
                    clm['Transaction_amount'].append(transaction_amount)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quater'].append(int(k.strip('.json')))

            except:
                pass   
                
map_transaction = pd.DataFrame(clm)
# Succesfully created a dataframe
# map_transaction.to_csv("phonepe_csv/map_transaction.csv")
map_transaction



,State,Year,Quater,District,Transaction_count,Transaction_amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
14037,west-bengal,2022,4,nadia district,12690126,2.804568e+10
14038,west-bengal,2022,4,birbhum district,7617444,1.614650e+10
14039,west-bengal,2022,4,purba medinipur district,14484229,3.309949e+10
14040,west-bengal,2022,4,maldah district,12492746,2.721861e+10


In [134]:
cursor.execute("create table map_transaction(state varchar(45),year int, quarter int,district_name varchar(45),transaction_count int, transaction_amount float)")

for record in map_transaction.values:
    cursor.execute("insert into map_transaction(state,year,quarter,district_name,transaction_count,transaction_amount)values{}".format(tuple(record)))
    connection.commit()
print('Data inserted into Table-3 successfully')
    
#**************************************** ------  3 rd finished  ------************************************************



Data inserted into Table-3 successfully


In [135]:
#                            4.  This is registered users in India :

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


path = "pulse/data/map/user/hover/country/india/state/"
state_list = os.listdir(path)
#Agg_state_list
# Agg_state_list--> to get the list of states in India

#<------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------>#

# This is to extract the data's to create a dataframe

clm = {'State': [], 'Year': [], 'Quater': [], 'District': [],
    'Registered_user': [], 'App_opening': []}
for i in state_list:
    p_i = path+i+"/"
    year = os.listdir(p_i)
    for j in year:
        p_j = p_i+j+"/"
        file = os.listdir(p_j)
        for k in file:
            p_k = p_j+k
            Data = open(p_k, 'r')
            D = json.load(Data)
            try:
                for z in D['data']["hoverData"]:
                    district = z
                    registered_user =  D['data']["hoverData"][z]["registeredUsers"]
                    app_opening = D['data']["hoverData"][z]["appOpens"]
                    clm['District'].append(district)
                    clm['Registered_user'].append(registered_user)
                    clm['App_opening'].append(app_opening)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quater'].append(int(k.strip('.json')))

            except:
                pass       
                 
registered_users = pd.DataFrame(clm)
# Succesfully created a dataframe
#registered_users.to_csv("phonepe_csv/district_registering.csv")

registered_users

,State,Year,Quater,District,Registered_user,App_opening
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
14635,west-bengal,2022,4,nadia district,1359420,33853990
14636,west-bengal,2022,4,birbhum district,855236,20950662
14637,west-bengal,2022,4,purba medinipur district,1346908,38278506
14638,west-bengal,2022,4,maldah district,954892,29023743


In [136]:
cursor.execute("create table registered_users(state varchar(45),year int,quarter int,district_name varchar(45), registered_users int, app_opens int)")
for record in district_registering.values:
    cursor.execute("insert into registered_users(state, year, quarter, district_name, registered_users, app_opens)values{}".format(tuple(record)))
    connection.commit()
print('Data inserted into Table-4 successfully')
    

#**************************************** ------  4 th finished  ------************************************************



Data inserted into Table-4 successfully


In [137]:
#                       5.  This is extract the data of top transactions made in all states:

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


path = 'pulse/data/top/transaction/country/india/state/'
state_list = os.listdir(path)
clm={'State':[],'Year':[],'Quarter':[],'district_name':[] ,'total_count':[],'total_amount':[]}

#--------------------------------------------------------------------------------------------------------------------------


for i in state_list:
    p_i= path + i + '/'
    year = os.listdir(p_i)
    for j in year:
        p_j = p_i + j + '/'
        file = os.listdir(p_j)
        for k in file:
            p_k = p_j + k 
            Data = open(p_k,'r')
            D = json.load(Data)
            for z in D['data']['districts']:
                    district_name = z['entityName']
                    count= z['metric']['count']
                    amount = z['metric']['amount']
                   
                    clm['district_name'].append(district_name)
                    clm['total_count'].append(count)
                    clm['total_amount'].append(amount)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quarter'].append(int(k.strip(".json")))

top_transac = pd.DataFrame(clm)
top_transac


,State,Year,Quarter,district_name,total_count,total_amount
0,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07
1,andaman-&-nicobar-islands,2018,1,nicobars,528,1.139849e+06
2,andaman-&-nicobar-islands,2018,1,north and middle andaman,442,9.316631e+05
3,andaman-&-nicobar-islands,2018,2,south andaman,9395,2.394824e+07
4,andaman-&-nicobar-islands,2018,2,nicobars,1120,3.072437e+06
...,...,...,...,...,...,...
5915,west-bengal,2022,4,south twenty four parganas,15651650,3.373698e+10
5916,west-bengal,2022,4,purba medinipur,14484229,3.309949e+10
5917,west-bengal,2022,4,hooghly,13931352,2.755409e+10
5918,west-bengal,2022,4,howrah,13350090,2.793786e+10


In [138]:
cursor.execute("create table top_transaction(state varchar(45),year int,quarter int,district_name varchar(45),transaction_count int,transaction_amount float)")
for record in top_transac.values:
    cursor.execute("insert into top_transaction(state,year,quarter,district_name,transaction_count,transaction_amount)values{}".format(tuple(record)))
    connection.commit()
print('Data inserted into Table-5 successfully')
    

#**************************************** ------  5 th finished  ------************************************************



Data inserted into Table-5 successfully


In [141]:
#                           6. This is to find the top users list in india:

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


path = 'pulse/data/top/user/country/india/state/'
state_list = os.listdir(path)
clm={'State':[],'Year':[],'Quarter':[],'district_name':[] ,'total_users':[]}


#---------------------------------------------------------------------------------------------------------------------------

for i in state_list:
    p_i= path + i + '/'
    year = os.listdir(p_i)
    for j in year:
        p_j = p_i + j + '/'
        file = os.listdir(p_j)
        for k in file:
            p_k = p_j + k 
            Data = open(p_k,'r')
            D = json.load(Data)
            for z in D['data']['districts']:
                    users = z['registeredUsers']
                    district_name = z['name']
                    clm['district_name'].append(district_name)
                    clm['total_users'].append(users)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quarter'].append(int(k.strip(".json")))

top_users = pd.DataFrame(clm)
top_users


,State,Year,Quarter,district_name,total_users
0,andaman-&-nicobar-islands,2018,1,south andaman,5846
1,andaman-&-nicobar-islands,2018,1,north and middle andaman,632
2,andaman-&-nicobar-islands,2018,1,nicobars,262
3,andaman-&-nicobar-islands,2018,2,south andaman,8143
4,andaman-&-nicobar-islands,2018,2,north and middle andaman,911
...,...,...,...,...,...
5915,west-bengal,2022,4,howrah,1422011
5916,west-bengal,2022,4,nadia,1359420
5917,west-bengal,2022,4,purba medinipur,1346908
5918,west-bengal,2022,4,paschim medinipur,1217113


In [142]:
cursor.execute("create table top_users(state varchar(45),year int,quarter int,district_name varchar(45),registered_users int)")             
for record in top_users.values:
    cursor.execute("insert into top_users(state, year, quarter, district_name, registered_users) values{}".format(tuple(record)))
    connection.commit()
print('Data inserted into Table-6 successfully')
    

#**************************************** ------  6 th finished  ------************************************************



Data inserted into Table-6 successfully


In [145]:
Agg_Transac 
userby_device 
map_transaction 
registered_users 
top_transac 
top_users


DeltaGenerator()

In [148]:
#webpage title
st.set_page_config(page_title="PhonPe_Pulse",layout="wide")
st.title("Data science Project")
#st.header(":violet[_PhonePe-Pulse_]")
new_title = '<p style="font-family:Courier; color:purple; font-size: 40px;">PhoePe-Pulse</p>'
st.markdown(new_title, unsafe_allow_html=True) 
left_column,right_column=st.columns(2)  

with left_column:
    option=st.selectbox(label="**Transactions/Users**",options=("Transaction","Users"))
    if option=='Transaction':
        transaction_data=st.selectbox(label="**Name Of Transaction**",options=("aggregate_transaction","map_transaction","top_transaction"))
        if transaction_data=='aggregate_transaction':
            type_transaction=st.selectbox(label="**Transaction Type**",options=("Recharge & bill payments","Peer-to-peer payments","Merchant payments","Financial Services","Others"))
            Quater=st.selectbox(label="**Quater**",options=("1","2","3","4"))
            year=st.selectbox("**Year**",range(2018,2023))
            query2="select state,year,Quater,tansaction_type ,sum(transaction_amount) from {} group by state,year,Quater,tansaction_type having year={} and Quater={} and tansaction_type='{}' ".format(transaction_data,year,Quater,type_transaction)
            df=pd.read_sql_query(query2, connection)
            with urlopen('https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson')as response:
                indian_states=json.load(response)
            list_states=[]
            for i in range(len(indian_states['features'])):
                state=indian_states['features'][i]['properties']['ST_NM']
                list_states.append(state)
            st_name=sorted(list_states)
            df['st_name']=st_name
            query="select sum(transaction_amount) from aggregate_transaction"
            total_trans=pd.read_sql_query(query, connection)
            with right_column:
                st.subheader("Total Transaction value")
                st.markdown(total_trans.values)
                st.subheader("{} value".format(type_transaction))
                st.markdown(df['sum(transaction_amount)'].sum())
                st.subheader("Average {} value".format(type_transaction))
                st.markdown(df['sum(transaction_amount)'].mean())
                data =px.choropleth(df,locations='st_name',
                                    title="Total {} of {}".format(transaction_data,type_transaction),
                                    geojson=indian_states,
                                    featureidkey='properties.ST_NM',
                                    color='sum(transaction_amount)',
                                    color_continuous_scale='Reds',
                                    scope='asia',
                                    projection="mercator",
                                    hover_data=['year','Quater']
                        
                                    )
                data.update_geos(fitbounds='locations', visible=False)
                st.plotly_chart(data,use_container_width=True)
        else:
            Quater=st.selectbox(label="**Quater**",options=("1","2","3","4"))
            year=st.selectbox("**Year**",range(2018,2023))
            query2="select state,year,Quater,sum(transaction_amount) from {} group by state,year,Quater having year={} and Quater={}".format(transaction_data,year,Quater)
            df=pd.read_sql_query(query2, connection)
            with urlopen('https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson')as response:
                indian_states=json.load(response)
            list_states=[]
            for i in range(len(indian_states['features'])):
                state=indian_states['features'][i]['properties']['ST_NM']
                list_states.append(state)
            st_name=sorted(list_states)
            df['st_name']=st_name
            with right_column:
                st.subheader("Total transaction value")
                st.markdown(df['sum(transaction_amount)'].sum())  
                st.subheader("Average transaction value")
                st.markdown(df['sum(transaction_amount)'].mean())
                data =px.choropleth(df,locations='st_name',
                                    title="Total {}".format(transaction_data),
                                    geojson=indian_states,
                                    featureidkey='properties.ST_NM',
                                    color='sum(transaction_amount)',
                                    color_continuous_scale='Reds',
                                    scope='asia',
                                    projection="mercator",
                                    hover_data=['year','Quater']
                                   
                 
                                    )
                data.update_geos(fitbounds='locations', visible=False)
                st.plotly_chart(data,use_container_width=True)
       
    elif option=='Users':
        users_data=st.selectbox(label="**Users Type**",options=("userby_device","registered_users","top_users"))
        if users_data=="userby_device":
            brand=st.selectbox(label="**Brand**",options=("Xiaomi","Samsung","Vivo","Oppo","OnePlus","Realme","Apple","Motorola","Lenovo","Huawei","Others"))
            Quater=st.selectbox(label="**Quater**",options=("1","2","3","4"))
            year=st.selectbox("**Year**",range(2018,2023))
            query2="select state,year,Quater,sum(users_count) from userby_device group by state,year,Quater having year={} and Quater={}".format(year,Quater)
            brand_query="select brand,sum(users_count) from userby_device group by brand"
            df=pd.read_sql_query(query2, connection)
            with urlopen('https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson')as response:
                indian_states=json.load(response)
            list_states=[]
            for i in range(len(indian_states['features'])):
                state=indian_states['features'][i]['properties']['ST_NM']
                list_states.append(state)
            st_name=sorted(list_states)
            df['st_name']=st_name
            with right_column:
                st.subheader("{} Users".format(brand))
                st.markdown(df['sum(users_count)'].sum())
                
                st.subheader("Total Registered Users")
                st.markdown(df['sum(users_count)'].sum())
                data =px.choropleth(df,locations='st_name',
                                    title="{}".format(users_data),
                                    geojson=indian_states,
                                    featureidkey='properties.ST_NM',
                                    color='sum(users_count)',
                                    color_continuous_scale='Reds',
                                    scope='asia',
                                    projection="mercator",
                                    hover_data=['year','Quater']
                                   
                 
                                    )
                data.update_geos(fitbounds='locations', visible=False)
                st.plotly_chart(data,use_container_width=True)
        else:
            Quater=st.selectbox(label="**Quater**",options=("1","2","3","4"))
            year=st.selectbox("**Year**",range(2018,2023))
            query2="select state,year,Quater,sum(registered_users) from {} group by state,year,Quater having year={} and Quater={}".format(users_data,year,Quater)
            df=pd.read_sql_query(query2, connection)
            with urlopen('https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson')as response:
                indian_states=json.load(response)
            list_states=[]
            for i in range(len(indian_states['features'])):
                state=indian_states['features'][i]['properties']['ST_NM']
                list_states.append(state)
            st_name=sorted(list_states)
            df['st_name']=st_name
            with right_column:
                st.subheader("Total Registered Users")
                st.markdown(df['sum(registered_users)'].sum())
                data =px.choropleth(df,locations='st_name',
                                    title="{}".format(users_data),
                                    geojson=indian_states,
                                    featureidkey='properties.ST_NM',
                                    color='sum(registered_users)',
                                    color_continuous_scale='Reds',
                                    scope='asia',
                                    projection="mercator",
                                    hover_data=['year','Quater']
                                   
                 
                                    )
                data.update_geos(fitbounds='locations', visible=False)
                st.plotly_chart(data,use_container_width=True)
                

C:\Users\rsiva\AppData\Local\Temp\ipykernel_5288\1699371791.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query2, connection)


NameError: name 'urlopen' is not defined

In [ ]:
state = state.sort_values(by='state')
state = state.reset_index(drop=True)
df2 = df.groupby(['State']).sum()[['Transacion_count', 'Transacion_amount']]
df2 = df2.reset_index()

choropleth_data = state.copy()

for column in df2.columns:
    choropleth_data[column] = df2[column]
choropleth_data = choropleth_data.drop(labels='State', axis=1)

df.rename(columns={'State': 'state'}, inplace=True)
sta_list = ['andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
            'assam', 'bihar', 'chandigarh', 'chhattisgarh',
            'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
            'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
            'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
            'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
            'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
            'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
            'uttarakhand', 'west-bengal']
state['state'] = pd.Series(data=sta_list)
state_final = pd.merge(df, state, how='outer', on='state')
districts_final = pd.merge(districts_tran, districts,
                           how='outer', on=['State', 'District'])


# ------------------------------------------ Streamlit app Plot 1 Scatter plot of registered user and app opening ----------------------------
st.balloons()
with st.container():
    st.title(':violet[PhonePe Pulse Data Visualization(2018-2022)📈]')
    st.image('data/Data-Vizualisation.png')
    st.write(' ')
    st.subheader(
        ':violet[Registered user & App installed -> State and Districtwise:]')
    st.write(' ')
    scatter_year = st.selectbox('Please select the Year',
                                ('2018', '2019', '2020', '2021', '2022'))
    st.write(' ')
    scatter_state = st.selectbox('Please select State', ('andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
                                                         'assam', 'bihar', 'chandigarh', 'chhattisgarh',
                                                         'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
                                                         'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
                                                         'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
                                                         'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
                                                         'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
                                                         'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
                                                         'uttarakhand', 'west-bengal'), index=10)
    scatter_year = int(scatter_year)
    scatter_reg_df = app_opening[(app_opening['Year'] == scatter_year) & (
        app_opening['State'] == scatter_state)]
    scatter_register = px.scatter(scatter_reg_df, x="District", y="Registered_user",  color="District",
                                  hover_name="District", hover_data=['Year', 'Quater', 'App_opening'], size_max=60)
    st.plotly_chart(scatter_register)
st.write(' ')


# ------------------------------------- Streamlit Tabs for various analysis -----------------------------------------------------------------
geo_analysis, Device_analysis, payment_analysis, transac_yearwise = st.tabs(
    ["Geographical analysis", "User device analysis", "Payment Types analysis", "Transacion analysis of States"])
# ------------------------------------------- Geo-analysis ----------------------------------------------------------------------------------
with geo_analysis:
    st.subheader(':violet[Transaction analysis->State and Districtwise:]')
    st.write(' ')
    Year = st.radio('Please select the Year',
                    ('2018', '2019', '2020', '2021', '2022'), horizontal=True)
    st.write(' ')
    Quarter = st.radio('Please select the Quarter',
                       ('1', '2', '3', '4'), horizontal=True)
    st.write(' ')
    Year = int(Year)
    Quarter = int(Quarter)
    plot_district = districts_final[(districts_final['Year'] == Year) & (
        districts_final['Quater'] == Quarter)]
    plot_state = state_final[(state_final['Year'] == Year)
                             & (state_final['Quater'] == Quarter)]
    plot_state_total = plot_state.groupby(
        ['state', 'Year', 'Quater', 'Latitude', 'Longitude']).sum()
    plot_state_total = plot_state_total.reset_index()
    state_code = ['AN', 'AD', 'AR', 'AS', 'BR', 'CH', 'CG', 'DNHDD', 'DL', 'GA',
                  'GJ', 'HR', 'HP', 'JK', 'JH', 'KA', 'KL', 'LA', 'LD', 'MP', 'MH',
                  'MN', 'ML', 'MZ', 'NL', 'OD', 'PY', 'PB', 'RJ', 'SK', 'TN', 'TS',
                  'TR', 'UP', 'UK', 'WB']
    plot_state_total['code'] = pd.Series(data=state_code)
    # ------------------------------------------- Geo-visualization of transacion data ------------------------------------------------------
    fig1 = px.scatter_geo(plot_district,
                          lon=plot_district['Longitude'],
                          lat=plot_district['Latitude'],
                          color=plot_district['Transaction_amount'],
                          size=plot_district['Transaction_count'],
                          hover_name="District",
                          hover_data=["State", 'Transaction_amount', 'Transaction_amount',
                                      'Transaction_count', 'Year', 'Quater'],
                          title='District',
                          size_max=22,)
    fig1.update_traces(marker={'color': "#CC0044",
                               'line_width': 1})
    fig2 = px.scatter_geo(plot_state_total,
                          lon=plot_state_total['Longitude'],
                          lat=plot_state_total['Latitude'],
                          hover_name='state',
                          text=plot_state_total['code'],
                          hover_data=['Transacion_count',
                                      'Transacion_amount', 'Year', 'Quater'],
                          )
    fig2.update_traces(marker=dict(color="#D5FFCC", size=0.3))
    fig = px.choropleth(
        choropleth_data,
        geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
        featureidkey='properties.ST_NM',
        locations='state',
        color='Transacion_amount',
        color_continuous_scale='twilight',
        hover_data=['Transacion_count', 'Transacion_amount']
    )

    fig.update_geos(fitbounds="locations", visible=False)
    fig.add_trace(fig1.data[0])
    fig.add_trace(fig2.data[0])
    fig.update_layout(height=1000, width=1000)
    st.write(' ')
    st.write(' ')
    if st.button('Click here to see map clearly'):
        fig.show(renderer="browser")
    st.plotly_chart(fig)


# --------------------------------------------------- Device analysis statewise ------------------------------------------------------------
with Device_analysis:
    st.subheader(':violet[User Device analysis->Statewise:]')
    tree_map_state = st.selectbox('Please select State', ('andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
                                                          'assam', 'bihar', 'chandigarh', 'chhattisgarh',
                                                          'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
                                                          'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
                                                          'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
                                                          'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
                                                          'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
                                                          'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
                                                          'uttarakhand', 'west-bengal'), index=10, key='tree_map_state')
    tree_map_state_year = int(st.radio('Please select the Year',
                                       ('2018', '2019', '2020', '2021', '2022'), horizontal=True, key='tree_map_state_year'))
    tree_map_state_quater = int(st.radio('Please select the Quarter',
                                         ('1', '2', '3', '4'), horizontal=True, key='tree_map_state_quater'))
    user_device_treemap = user_device[(user_device['State'] == tree_map_state) & (user_device['Year'] == tree_map_state_year) &
                                      (user_device['Quater'] == tree_map_state_quater)]
    user_device_treemap['Brand_count'] = user_device_treemap['Brand_count'].astype(
        str)
    # ----------------------------------------- Treemap view of user device ----------------------------------------------------------------
    user_device_treemap_fig = px.treemap(user_device_treemap, path=['State', 'Brand'], values='Brand_percentage', hover_data=['Year', 'Quater'],
                                         color='Brand_count',
                                         title='User device distribution in ' + tree_map_state +
                                         ' in ' + str(tree_map_state_year)+' at '+str(tree_map_state_quater)+' quater',)
    st.plotly_chart(user_device_treemap_fig)
    # ---------------------------------------- Barchart view of user device -----------------------------------------------------------------
    bar_user = px.bar(user_device_treemap, x='Brand', y='Brand_count', color='Brand',
                      title='Bar chart analysis', color_continuous_scale='sunset',)
    st.plotly_chart(bar_user)


# ----------------------------------------- Payment type analysis of Transacion data ----------------------------------------------------------
with payment_analysis:
    st.subheader(':violet[Payment type Analysis -> 2018 - 2022:]')
    querypa = 'select * from agg_transaction_table'
    payment_mode = pd.read_sql(querypa, con=connection)
    #payment_mode = pd.read_csv('csv/Agg_Trans.csv', index_col=0)
    pie_pay_mode_state = st.selectbox('Please select State', ('andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
                                                              'assam', 'bihar', 'chandigarh', 'chhattisgarh',
                                                              'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
                                                              'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
                                                              'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
                                                              'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
                                                              'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
                                                              'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
                                                              'uttarakhand', 'west-bengal'), index=10, key='pie_pay_mode_state')
    pie_pay_mode_year = int(st.radio('Please select the Year',
                                     ('2018', '2019', '2020', '2021', '2022'), horizontal=True, key='pie_pay_year'))
    pie_pay_mode__quater = int(st.radio('Please select the Quarter',
                                        ('1', '2', '3', '4'), horizontal=True, key='pie_pay_quater'))
    pie_pay_mode_values = st.selectbox(
        'Please select the values to visualize', ('Transacion_count', 'Transacion_amount'))
    pie_payment_mode = payment_mode[(payment_mode['Year'] == pie_pay_mode_year) & (
        payment_mode['Quater'] == pie_pay_mode__quater) & (payment_mode['State'] == pie_pay_mode_state)]
    # -------------------------------- Pie chart analysis of Payment mode --------------------------------------------------------------------
    pie_pay_mode = px.pie(pie_payment_mode, values=pie_pay_mode_values,
                          names='Transacion_type', hole=.5, hover_data=['Year'])
    # ------------------------------------- Bar chart analysis of payment mode ----------------------------------------------------------------
    pay_bar = px.bar(pie_payment_mode, x='Transacion_type',
                     y=pie_pay_mode_values, color='Transacion_type')
    st.plotly_chart(pay_bar)
    st.plotly_chart(pie_pay_mode)


# --------------------------------------- Transacion data analysis statewise ------------------------------------------------------------------
with transac_yearwise:
    st.subheader(':violet[Transaction analysis->Statewise:]')
    transac_state = st.selectbox('Please select State', ('andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh',
                                                         'assam', 'bihar', 'chandigarh', 'chhattisgarh',
                                                         'dadra-&-nagar-haveli-&-daman-&-diu', 'delhi', 'goa', 'gujarat',
                                                         'haryana', 'himachal-pradesh', 'jammu-&-kashmir', 'jharkhand',
                                                         'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
                                                         'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland',
                                                         'odisha', 'puducherry', 'punjab', 'rajasthan', 'sikkim',
                                                         'tamil-nadu', 'telangana', 'tripura', 'uttar-pradesh',
                                                         'uttarakhand', 'west-bengal'), index=10, key='transac')
    transac__quater = int(st.radio('Please select the Quarter',
                                   ('1', '2', '3', '4'), horizontal=True, key='trans_quater'))
    transac_type = st.selectbox('Please select the Mode',
                                ('Recharge & bill payments', 'Peer-to-peer payments', 'Merchant payments', 'Financial Services', 'Others'), key='transactype')
    transac_values = st.selectbox(
        'Please select the values to visualize', ('Transacion_count', 'Transacion_amount'), key='transacvalues')
    #payment_mode_yearwise = pd.read_csv('csv/Agg_Trans.csv', index_col=0)

    querypay_year = 'select * from agg_transaction_table'
    payment_mode_yearwise = pd.read_sql(querypay_year, con=connection)

    new_df = payment_mode_yearwise.groupby(
        ['State', 'Year', 'Quater', 'Transacion_type']).sum()
    new_df = new_df.reset_index()
    chart = new_df[(new_df['State'] == transac_state) &
                   (new_df['Transacion_type'] == transac_type) & (new_df['Quater'] == transac__quater)]
    # ------------------------------- Bar chart analysis of transacion data statewise --------------------------------------------------------
    year_fig = px.bar(chart, x=['Year'], y=transac_values, color=transac_values, color_continuous_scale='armyrose',
                      title='Transacion analysis '+transac_state + ' regarding to '+transac_type)
    st.plotly_chart(year_fig)


# -------------------------------------------- Sidebar --> for overall india Data comparisons -------------------------------------------------
with st.sidebar:
    # -------------------------- Bar chart ofoverall india transacion data  -----------------------------------------------------------------
    st.subheader(':violet[Overall India Analysis:]')
    overall_values = st.selectbox(
        'Please select the values to visualize', ('Transacion_count', 'Transacion_amount'), key='values')
    overall = new_df.groupby(['Year']).sum()
    overall.reset_index(inplace=True)

    overall = px.bar(overall, x='Year', y=overall_values, color=overall_values,
                     title='Overall pattern of Transacion all over India', color_continuous_scale='sunset',)
    overall.update_layout(height=350, width=350)
    st.plotly_chart(overall)

    # --------------------------Bar chart of overall india user device analysis --------------------------------------------------------------
    query_device = 'select * from agg_userbydevice_table'
    user_device_overall = pd.read_sql(query_device, con=connection)
    #user_device_overall = pd.read_csv('csv/user_by_device.csv', index_col=0)
    overall_device = user_device_overall.groupby(['Brand', 'Year']).sum()
    overall_device.reset_index(inplace=True)

    overall_dev_fig = px.bar(overall_device, x='Year', y='Brand_count',
                             color='Brand', title='Customer Device pattern from 2018 - 2022')
    overall_dev_fig.update_layout(height=350, width=350)
    st.plotly_chart(overall_dev_fig)

    # --------------------------Bar chart of overall india registered and app opening --------------------------------------------------------
    query_reg = 'select * from district_map_registering_table'
    overall_reg = pd.read_sql(query5, con=connection)
    #overall_reg = pd.read_csv('csv/district_registering_map.csv')
    overall_reg = overall_reg.groupby(['State', 'Year']).sum()
    overall_reg.reset_index(inplace=True)

    overall_reg = px.bar(overall_reg, x='Year', y=[
                         'Registered_user', 'App_opening'], barmode='group', title='Phonepe installation from 2018 - 2022')
    overall_reg.update_layout(height=350, width=350)
    st.plotly_chart(overall_reg)